In [3]:
import torch
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from datasets import load_metric
from sklearn.model_selection import train_test_split
import torch.nn as nn

# Load dataset from CSV
df = pd.read_csv("../data/raw/merged_email_data.csv")

# Custom dataset class
class EmailDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512, summary_length=128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length
        self.summary_length = summary_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        thread = item['body']  # Assuming 'body' contains the email threads
        summary = item['summary']
        
        model_input = self.tokenizer(thread, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(summary, max_length=self.summary_length, truncation=True, padding='max_length', return_tensors="pt")
        
        model_input["labels"] = labels["input_ids"].squeeze()
        
        return {key: val.squeeze() for key, val in model_input.items()}

# Initialize tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Split dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Prepare train and validation datasets
train_dataset = EmailDataset(tokenizer, train_df)
val_dataset = EmailDataset(tokenizer, val_df)

# Initialize DataLoader for train and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Increased batch size for faster training
val_dataloader = DataLoader(val_dataset, batch_size=8)  # Increased batch size for faster evaluation

# Initialize BART model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
bart_model.to(device)

# Evaluation function with ROUGE scores
def evaluate(model, dataloader):
    rouge = load_metric("rouge")
    model.eval()
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            generated_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in generated_ids]
            references = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in labels]
            
            rouge.add_batch(predictions=preds, references=references)

    result = rouge.compute()
    return {key: value.mid.fmeasure * 100 for key, value in result.items()}

# Evaluate before fine-tuning
print("Evaluating before fine-tuning...")
before_scores = evaluate(bart_model, val_dataloader)
print(before_scores)

# Prepare optimizer and scheduler
optimizer = AdamW(bart_model.parameters(), lr=5e-5)
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Fine-tuning
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    bart_model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = bart_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

# Evaluate after fine-tuning
print("Evaluating after fine-tuning...")
after_scores = evaluate(bart_model, val_dataloader)
print(after_scores)

print("Model evaluation and fine-tuning complete.")


# prompt: write code to save above model
# Save the fine-tuned model
torch.save(bart_model.state_dict(), "models/email_summarizer_bart.pth")

Evaluating before fine-tuning...


/var/folders/nk/vpkrtrv925b_11h3jrsctp8w0000gn/T/ipykernel_46398/2014295430.py:58: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/Users/abhishekmurthy/Code/Duke/540/EmailSummarization/env/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/Users/abhishekmurthy/Code/Duke/540/EmailSummarization/env/lib/python3.9/site-packages/transformers/tokenization_utils_

KeyboardInterrupt: 